In [1]:
import torch
import torch.nn.functional as F

# General Memory Functions

In [2]:
x = torch.randn(3, 3)

In [3]:
x.element_size()

4

In [4]:
x.numel()

9

In [5]:
def sizeof_fmt(num, suffix="B"):
    for unit in ["", "Ki", "Mi", "Gi", "Ti", "Pi", "Ei", "Zi"]:
        if abs(num) < 1024.0:
            return f"{num:3.1f}{unit}{suffix}"
        num /= 1024.0
    return f"{num:.1f}Yi{suffix}"


def mem_size(t: torch.Tensor):
    return sizeof_fmt(t.element_size() * t.numel())


In [6]:
mem_size(x)

'36.0B'

# RAM Use

Resting RAM Use 2.02gb (jupyter, OS, other random stuff)

In [7]:
# RAM jumps up here to 5.76gb
t1 = torch.randn((100000, 10000))
mem_size(t1)

'3.7GiB'

In [8]:
# RAM jumps up to ~9.3gb, until execution completes and then drops
# to 5.76gb again
# memory of previous variable not released until new allocation 
# complete (or the GC is a bit slow)
t1 = torch.randn((100000, 10000))

In [10]:
# Drops to 2.02gb, after GC
t1 = None

In [11]:
# Getting back to 5.76gb
t1 = torch.randn((100000, 10000))

In [16]:
# Memory drops before rising again
t1 = None
t1 = torch.randn((100000, 10000))

In [18]:
# Could also force a gc with:
import gc
t1 = None
gc.collect()

0

# VRAM Use

Investigating VRAM usage on a AMD GPU, with ROCm 5.4.3.50403-121~20.04

In [ ]:
t1 = 